In [16]:
import sys
from typing import Literal

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
n_samples = 32
sampler = EquationSampler(BASE_TOKENS, 16, 2)
sequences, entropies, log_probs = sampler.sample_sequence(n_samples, repeat=5)

In [9]:
shape = (n_samples, 0)
counters = tf.ones(n_samples)
lengths = tf.zeros(n_samples)

In [18]:
sequences

<tf.Tensor: shape=(32, 5), dtype=int32, numpy=
array([[4, 7, 6, 3, 2],
       [6, 4, 1, 5, 4],
       [4, 5, 0, 4, 9],
       [3, 4, 4, 4, 3],
       [1, 4, 3, 3, 4],
       [1, 1, 4, 3, 7],
       [4, 3, 3, 1, 5],
       [7, 5, 1, 7, 6],
       [4, 5, 7, 6, 3],
       [4, 1, 4, 4, 2],
       [3, 3, 3, 7, 2],
       [3, 9, 4, 3, 3],
       [4, 4, 4, 4, 4],
       [5, 6, 3, 2, 4],
       [7, 2, 4, 5, 0],
       [4, 7, 3, 1, 4],
       [3, 4, 3, 3, 7],
       [4, 4, 4, 7, 3],
       [0, 3, 2, 3, 3],
       [1, 3, 3, 5, 4],
       [3, 3, 1, 4, 4],
       [1, 4, 3, 3, 1],
       [3, 0, 3, 4, 4],
       [4, 3, 1, 7, 4],
       [1, 3, 3, 7, 0],
       [7, 7, 5, 3, 4],
       [4, 3, 2, 0, 5],
       [1, 2, 2, 2, 4],
       [4, 7, 4, 4, 0],
       [4, 4, 0, 3, 5],
       [4, 3, 2, 3, 2],
       [7, 1, 9, 7, 1]], dtype=int32)>